In [1]:
import xgboost as xgb
import pandas as pd
import numpy as np
from datetime import timedelta
from pprint import pprint
import dataframe_image as dfi
import matplotlib.pyplot as plt
import sklearn.metrics as mtr
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.inspection import permutation_importance
import pickle

pd.options.display.max_colwidth = 200

In [2]:
df = pd.read_csv("../Bases/Cleaned/rota32893.csv")

df["data_partida"] = pd.to_datetime(df["data_partida"])

df = df.sort_values(by=["data_partida"], ascending=True)

df["tempo_viagem_1"] = df["tempo_viagem"]
df["tempo_viagem_2"] = df["tempo_viagem"]
df["tempo_viagem_3"] = df["tempo_viagem"]

df["tempo_viagem_1"] = df["tempo_viagem_1"].shift(1)
df["tempo_viagem_2"] = df["tempo_viagem_2"].shift(2)
df["tempo_viagem_3"] = df["tempo_viagem_3"].shift(3)

df.to_csv('../Bases/Cleaned/rota32893_bf_times.csv',index=False)
df

,data_partida,tempo_viagem,dia_semana,partidaTimeStamp,qtdDiasAno,tipo_dia,turno_dia,hora_dia,tempo_viagem_1,tempo_viagem_2,tempo_viagem_3
5271,2019-07-01 05:49:50,2335,1,1561970990,182,1,2,5,NaN,NaN,NaN
5270,2019-07-01 06:05:57,2343,1,1561971957,182,1,2,6,2335.0,NaN,NaN
5269,2019-07-01 06:47:57,2163,1,1561974477,182,1,2,6,2343.0,2335.0,NaN
5268,2019-07-01 07:06:04,2912,1,1561975564,182,1,2,7,2163.0,2343.0,2335.0
5267,2019-07-01 07:27:18,2123,1,1561976838,182,1,2,7,2912.0,2163.0,2343.0
...,...,...,...,...,...,...,...,...,...,...,...
4,2019-09-30 21:38:06,1590,1,1569890286,273,1,4,21,1634.0,1904.0,1795.0
3,2019-09-30 21:50:20,1867,1,1569891020,273,1,4,21,1590.0,1634.0,1904.0
2,2019-09-30 22:30:04,1453,1,1569893404,273,1,4,22,1867.0,1590.0,1634.0
1,2019-09-30 22:40:17,1671,1,1569894017,273,1,4,22,1453.0,1867.0,1590.0


In [3]:
best_models = None
best_metric = None
best_day = ['14_79','14_79','14_79','14_79']
best_cenario = [0,0,0,0]

In [4]:
data_atual = df["data_partida"].dt.date.min() + timedelta(days=14)
dias_teste = 14
dias_treino = 79

while data_atual < (df["data_partida"].dt.date.max() - timedelta(days=14)):
        df_teste = df[df["data_partida"].dt.date > data_atual]
        df_treino = df[df["data_partida"].dt.date <= data_atual]

        for cenario in range(0, 4):
                if cenario == 0:
                        x_teste = df_teste[["partidaTimeStamp", "dia_semana", "tipo_dia", "hora_dia"]]
                        y_teste = df_teste["tempo_viagem"]
                        x_treino = df_treino[["partidaTimeStamp", "dia_semana", "tipo_dia", "hora_dia"]]
                        y_treino = df_treino["tempo_viagem"]

                elif cenario == 1:
                        df_teste = df_teste.iloc[1:]
                        df_treino = df_treino.iloc[1:]

                        x_teste = df_teste[["partidaTimeStamp", "dia_semana", "tipo_dia", "hora_dia", "tempo_viagem_1"]]
                        y_teste = df_teste["tempo_viagem"]
                        x_treino = df_treino[["partidaTimeStamp", "dia_semana", "tipo_dia", "hora_dia", "tempo_viagem_1"]]
                        y_treino = df_treino["tempo_viagem"]
                
                elif cenario == 2:
                        df_teste = df_teste.iloc[2:]
                        df_treino = df_treino.iloc[2:]

                        x_teste = df_teste[["partidaTimeStamp", "dia_semana", "tipo_dia", "hora_dia", "tempo_viagem_1", "tempo_viagem_2"]]
                        y_teste = df_teste["tempo_viagem"]
                        x_treino = df_treino[["partidaTimeStamp", "dia_semana", "tipo_dia", "hora_dia", "tempo_viagem_1", "tempo_viagem_2"]]
                        y_treino = df_treino["tempo_viagem"]
                
                else:
                        df_teste = df_teste.iloc[3:]
                        df_treino = df_treino.iloc[3:]

                        x_teste = df_teste[["partidaTimeStamp", "dia_semana", "tipo_dia", "hora_dia", "tempo_viagem_1", "tempo_viagem_2", "tempo_viagem_3"]]
                        y_teste = df_teste["tempo_viagem"]
                        x_treino = df_treino[["partidaTimeStamp", "dia_semana", "tipo_dia", "hora_dia", "tempo_viagem_1", "tempo_viagem_2", "tempo_viagem_3"]]
                        y_treino = df_treino["tempo_viagem"]
                

                # Hyperparâmetros do XGBoosting
                params_xgb = {
                        "n_estimators": list(range(100, 1100, 100)), #Number of gradient boosted trees. Equivalent to number of boosting rounds
                        "max_depth": list(range(2, 15)),#Maximum tree depth for base learners.
                        "min_child_weight": list(range(1, 11)),#Minimum sum of instance weight(hessian) needed in a child.
                        "learning_rate": [0.3, 0.2, 0.1, 0.05, 0.01, 0.005],#Boosting learning rate (xgb’s “eta”)
                        "gamma": np.arange(0, 0.7, 0.1)#Minimum loss reduction required to make a further partition on a leaf node of the tree.
                        }

                # Hyperparâmetros do Random Forest
                params_rf = {
                        "n_estimators": list(range(100, 1100, 100)),#The number of trees in the forest.
                        "bootstrap": [True, False],#Whether bootstrap samples are used when building trees. If False, the whole dataset is used to build each tree.
                        "max_depth": list(range(2, 15)),#The maximum depth of the tree.
                        "max_features": [1.0, "sqrt", "log2"],#The number of features to consider when looking for the best split:
                        "min_samples_leaf": list(range(1, 11)),#The minimum number of samples required to split an internal node
                        "min_samples_split": list(range(2, 11)),#The minimum number of samples required to be at a leaf node.
                        }

                # Hyperparâmetros do SVR
                params_svr = {
                        "C": [0.1, 1, 10, 100, 1000], #Regularization parameter. The strength of the regularization is inversely proportional to C
                        "gamma": ["scale", "auto"],#Kernel coefficient for ‘rbf’, ‘poly’ and ‘sigmoid’.
                        }

                #Linear Regression não tem hyper parametrização


                #Criação e fit do modelo XGB
                modelo_xgb = xgb.XGBRegressor(early_stop_rounds = 100)
                xgb_rand_search = RandomizedSearchCV(modelo_xgb, params_xgb, scoring="neg_mean_squared_error", n_iter=40, verbose=True, cv=10, n_jobs=-1, random_state=123)
                xgb_rand_search.fit(x_treino, y_treino)
                modelo_xgb = xgb_rand_search.best_estimator_

                modelo_xgb.fit(x_treino, y_treino, eval_set = [(x_treino, y_treino)])


                #Criação e fit do modelo Random Forest
                modelo_rf = RandomForestRegressor()
                rf_rand_search = RandomizedSearchCV(modelo_rf, params_rf, scoring="neg_mean_squared_error", n_iter=40, verbose=True, cv=10, n_jobs=-1, random_state=123)
                rf_rand_search.fit(x_treino, y_treino)
                modelo_rf = rf_rand_search.best_estimator_

                modelo_rf.fit(x_treino, y_treino)


                #Criação e fit do modelo SVR
                modelo_svr = SVR(kernel = 'rbf')
                svr_rand_search = RandomizedSearchCV(modelo_svr, params_svr, scoring="neg_mean_squared_error", n_iter=10, verbose=True, cv=10, n_jobs=-1, random_state=123)
                svr_rand_search.fit(x_treino, y_treino)
                modelo_svr = svr_rand_search.best_estimator_

                modelo_svr.fit(x_treino, y_treino)


                #Criação e fit do modelo Linear Regression
                modelo_lr = LinearRegression()
                modelo_lr.fit(x_treino, y_treino)

                df_teste["predicao_xgb"] = modelo_xgb.predict(x_teste)
                df_teste["predicao_lr"] = modelo_lr.predict(x_teste)
                df_teste["predicao_rf"] = modelo_rf.predict(x_teste)
                df_teste["predicao_svr"] = modelo_svr.predict(x_teste)

                df_teste.to_csv(f"../Bases/Predictions/rota32893_predictions_{dias_teste}_{dias_treino}_cen_{cenario}.csv",index=False)

                RMSE_xgb = mtr.mean_squared_error(df_teste["tempo_viagem"], df_teste["predicao_xgb"], squared=False) 
                RMSE_rf = mtr.mean_squared_error(df_teste["tempo_viagem"], df_teste["predicao_rf"], squared=False)       
                RMSE_svr = mtr.mean_squared_error(df_teste["tempo_viagem"], df_teste["predicao_svr"], squared=False) 
                RMSE_lr = mtr.mean_squared_error(df_teste["tempo_viagem"], df_teste["predicao_lr"], squared=False) 

                models = [modelo_xgb, modelo_rf, modelo_svr, modelo_lr]
                metrics = [RMSE_xgb,RMSE_rf,RMSE_svr,RMSE_lr]

                if best_models is None:
                        best_models = models
                        best_metrics = metrics
                else:
                        for i in range(4):
                                if best_metrics[i] > metrics[i]:
                                        best_models[i] = models[i]
                                        best_metrics[i] = metrics[i]
                                        best_day[i] = f'{dias_teste}_{dias_treino}'
                                        best_cenario[i] = cenario


        dias_teste += 14
        dias_treino -= 14

        data_atual += timedelta(days=14)

Fitting 10 folds for each of 40 candidates, totalling 400 fits
[09:20:46] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "early_stop_rounds" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[09:20:46] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "early_stop_rounds" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-rmse:2999.94383
[1]	validation_0-rmse:2986.63078
[2]	validation_0-rm

C:\Users\Thiago\AppData\Local\Temp\ipykernel_4396\4145900960.py:103: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teste["predicao_xgb"] = modelo_xgb.predict(x_teste)
C:\Users\Thiago\AppData\Local\Temp\ipykernel_4396\4145900960.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teste["predicao_lr"] = modelo_lr.predict(x_teste)
C:\Users\Thiago\AppData\Local\Temp\ipykernel_4396\4145900960.py:105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .lo

Fitting 10 folds for each of 40 candidates, totalling 400 fits
[09:21:37] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "early_stop_rounds" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[09:21:37] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "early_stop_rounds" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-rmse:3000.61208
[1]	validation_0-rmse:2987.16624
[2]	validation_0-rm

C:\Users\Thiago\AppData\Local\Temp\ipykernel_4396\4145900960.py:103: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teste["predicao_xgb"] = modelo_xgb.predict(x_teste)
C:\Users\Thiago\AppData\Local\Temp\ipykernel_4396\4145900960.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teste["predicao_lr"] = modelo_lr.predict(x_teste)
C:\Users\Thiago\AppData\Local\Temp\ipykernel_4396\4145900960.py:105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .lo

Fitting 10 folds for each of 40 candidates, totalling 400 fits
[09:22:34] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "early_stop_rounds" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[09:22:34] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "early_stop_rounds" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-rmse:3002.37619
[1]	validation_0-rmse:2988.91662
[2]	validation_0-rm

...

C:\Users\Thiago\AppData\Local\Temp\ipykernel_4396\4145900960.py:103: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teste["predicao_xgb"] = modelo_xgb.predict(x_teste)
C:\Users\Thiago\AppData\Local\Temp\ipykernel_4396\4145900960.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teste["predicao_lr"] = modelo_lr.predict(x_teste)
C:\Users\Thiago\AppData\Local\Temp\ipykernel_4396\4145900960.py:105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .lo

Fitting 10 folds for each of 40 candidates, totalling 400 fits
[09:56:59] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "early_stop_rounds" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[09:57:00] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "early_stop_rounds" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-rmse:2892.71609
[1]	validation_0-rmse:2879.55403
[2]	validation_0-rm

C:\Users\Thiago\AppData\Local\Temp\ipykernel_4396\4145900960.py:103: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teste["predicao_xgb"] = modelo_xgb.predict(x_teste)
C:\Users\Thiago\AppData\Local\Temp\ipykernel_4396\4145900960.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teste["predicao_lr"] = modelo_lr.predict(x_teste)
C:\Users\Thiago\AppData\Local\Temp\ipykernel_4396\4145900960.py:105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .lo

Fitting 10 folds for each of 40 candidates, totalling 400 fits
[10:01:01] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "early_stop_rounds" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[10:01:02] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "early_stop_rounds" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-rmse:2892.98065
[1]	validation_0-rmse:2879.81545
[2]	validation_0-rm

C:\Users\Thiago\AppData\Local\Temp\ipykernel_4396\4145900960.py:103: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teste["predicao_xgb"] = modelo_xgb.predict(x_teste)
C:\Users\Thiago\AppData\Local\Temp\ipykernel_4396\4145900960.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teste["predicao_lr"] = modelo_lr.predict(x_teste)
C:\Users\Thiago\AppData\Local\Temp\ipykernel_4396\4145900960.py:105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .lo

In [5]:
models_name = ['xgb', 'rf', 'svr', 'lr']
for i in range(4):
    with open (f'ModelsF2/rota32893_{models_name[i]}_{best_day[i]}_cen_{best_cenario[i]}.pkl', 'wb') as m:
        pickle.dump(best_models[i], m)